In [1]:
import pandas as pd
import numpy as np
import dataframe_image as dfi
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from scipy.stats import chisquare
from scipy.stats import mannwhitneyu
from itertools import combinations
from math import log, log10, exp, e, pow, sqrt
from statsmodels.sandbox.stats.multicomp import multipletests
import seaborn as sns
from statannotations.Annotator import Annotator
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# p value signifigance
def get_asterisks_for_pval(p_val):
    """Receives the p-value and returns asterisks string."""
    if p_val > 0.05:
        p_text = "ns"  # above threshold => not significant
    elif p_val < 1e-4:  
        p_text = '****'
    elif p_val < 1e-3:
        p_text = '***'
    elif p_val < 1e-2:
        p_text = '**'
    else:
        p_text = '*'
    
    return p_text

In [2]:
def chisq_and_posthoc_corrected():
# define dataset as dff
    contigency= pd.crosstab(dff[x], dff['Flu pos']).round(3)
    print(contigency)
    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(contigency, correction=True)
    print(f"Chi2 result of the IAV contingency table: {chi2}, p-value: {p} ({get_asterisks_for_pval(p)})")
    print("")

    # post-hoc
    all_combinations = list(combinations(contigency.index, 2))  # gathering all combinations for post-hoc chi2
    p_vals = []
    print("Corrected multiple comparisons:")
    for comb in all_combinations:
        new_df = contigency[(contigency.index == comb[0]) | (contigency.index == comb[1])]
        chi2, p, dof, ex = chi2_contingency(new_df, correction=True)
        p_vals.append(p)
        # print(f"For {comb}: {p}")  # uncorrected

    # checking significance
    # correction for multiple testing
    reject_list, corrected_p_vals = multipletests(p_vals, method='fdr_bh')[:2]
    for p_val, corr_p_val, reject, comb in zip(p_vals, corrected_p_vals, reject_list, all_combinations):
        print(f"{comb}: p_value: {p_val:5f}; corrected: {corr_p_val:5f} ({get_asterisks_for_pval(p_val)}) reject: {reject}")


In [3]:
def chisq_test():
# define dataset as dff
# define parameters as x and y
    contigency= pd.crosstab(dff[x], dff[y]).round(3)
    print(contigency)
    # start by running chi2 test on the matrix
    chi2, p, dof, ex = chi2_contingency(contigency, correction=True)
    print(f"Chi2 result of the IAV contingency table: {chi2}, p-value: {p} ({get_asterisks_for_pval(p)})")
    print("")

In [4]:
def generate_distribution_histogram(dataframe, 
                                    column_name, 
                                    title, x_axis_label, y_axis_label,
                                    label_name,
                                    number_bins = 15):
    """
    This function generates a histogram.
    Args:
        dataframe:
        column_name: String. Name of the column whose distribution we
        want to visualize.
        title: String. Title of the histogram.
        x_axis_label: String. X-axis label.
        y_axis_label: String. Y-axis label.
    Outputs:
        Histogram containing distribution for specific column column_name.
    """
    plt.hist(dataframe[column_name], bins = number_bins, label = label_name)
    plt.title(title)
    plt.xlabel(x_axis_label)
    plt.ylabel(y_axis_label)
    plt.legend(loc='upper right')

def mann_whitney_u_test(distribution_1, distribution_2):
    """
    Perform the Mann-Whitney U Test, comparing two different distributions.
    Args:
       distribution_1: List. 
       distribution_2: List.
    Outputs:
        u_statistic: Float. U statisitic for the test.
        p_value: Float.
    """

    u_statistic, p_val = mannwhitneyu(distribution_1, distribution_2)
    print('Mann-Whitney U test for difference in distribution results:')
    print(f'p-value: {p_val} ({get_asterisks_for_pval(p_val)})')
    print(f'U statistic: {u_statistic}')
    return 

In [5]:
# Original Dataset, dropping excess rows/columns
df = pd.read_csv('Country_age_strat.csv', encoding='latin-1')
df_agg = pd.read_csv('Country_agg.csv', encoding='latin-1')
df.dropna(how='all', inplace=True)
df_agg.dropna(how='all', inplace=True)

In [6]:
df

,country,date,age_group,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,...,% female infection,% male infection,% female death,% male death,Country begin,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total
0,Afghanistan,11/16/2021,19-Oct,10.0,19.0,13.0,12.0,4792.0,4792000,5030.0,...,48.79,51.21,52.00,48.00,Afghanistan,1,0,0,0,0
1,Afghanistan,11/16/2021,20-29,20.0,29.0,41.0,66.0,3408.0,3408000,3628.0,...,48.44,51.56,38.32,61.68,NaN,0,1,0,0,0
2,Afghanistan,11/16/2021,30-39,30.0,39.0,55.0,142.0,2167.0,2167000,2367.0,...,47.79,52.21,27.92,72.08,NaN,0,1,0,0,0
3,Afghanistan,11/16/2021,40-49,40.0,49.0,141.0,291.0,1418.0,1418000,1545.0,...,47.86,52.14,32.64,67.36,NaN,0,1,0,0,0
4,Afghanistan,11/16/2021,50-59,50.0,59.0,318.0,472.0,900.0,900000,941.0,...,48.89,51.11,40.25,59.75,NaN,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Wales,6/16/2022,45-64,45.0,64.0,413.0,612.0,426.0,426000,403.0,...,51.39,48.61,40.29,59.71,NaN,0,0,0,0,0
1459,Wales,6/16/2022,65-74,65.0,74.0,650.0,1064.0,187.0,187000,175.0,...,51.66,48.34,37.92,62.08,NaN,0,0,1,0,0
1460,Wales,6/16/2022,75-84,75.0,84.0,1349.0,1971.0,120.0,120000,102.0,...,54.05,45.95,40.63,59.37,NaN,0,0,0,0,0
1461,Wales,6/16/2022,85+,85.0,999.0,2244.0,1823.0,53.0,53000,32.0,...,62.35,37.65,55.18,44.82,NaN,0,0,0,1,0


In [7]:
df_agg

,country,deathsF,deathsM,populationin1000sF,populationin1000sM,Total infection,Total Death,% female infection,% male infection,% female death,% male death,Num female infection,Num female death,Num male infection,Num male death
0,Afghanistan,1375,2574,13545,14295,27840000,3949,48.65,51.35,34.82,65.18,13544160,1375,14295840,2574
1,Albania,703,1320,1413,1463,2876000,2023,49.13,50.87,34.75,65.25,1412979,703,1463021,1320
2,Argentina,50676,68751,23148,22048,45196000,119229,51.22,48.78,42.50,57.66,23149391,50672,22046609,68747
3,Aruba,68,83,57,52,109000,151,52.29,47.71,45.03,54.97,56996,68,52004,83
4,Australia,3628,4811,12801,12698,25499000,8439,50.20,49.80,42.99,57.01,12800498,3628,12698502,4811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Uganda,112,252,15838,15046,30884000,364,51.28,48.72,30.77,69.23,15837315,112,15046685,252
102,Ukraine,57678,48780,23472,20264,43736000,106458,53.67,46.33,54.18,45.82,23473111,57679,20262889,48779
103,Venezuela,236,365,14452,13986,28438000,601,50.82,49.18,39.27,60.73,14452192,236,13985808,365
104,Vietnam,49,37,48741,48153,96894000,86,50.30,49.70,56.98,43.02,48737682,49,48156318,37


In [8]:
df.columns

Index(['country', 'date', 'age_group', 'age_begin', 'age_end', 'deathsF',
       'deathsM', 'populationin1000sF', 'Num female infection',
       'populationin1000sM', 'Num male infection', 'Total infection',
       'Total Death', '% female infection', '% male infection',
       '% female death', '% male death', 'Country begin', 'Bin 0-19',
       'Bin 20-49', 'Bin 50-79', 'Bin 80+', 'Total'],
      dtype='object')

In [9]:
df_agg.columns

Index(['country', 'deathsF', 'deathsM', 'populationin1000sF',
       'populationin1000sM', 'Total infection', 'Total Death',
       '% female infection', '% male infection', '% female death',
       '% male death', 'Num female infection', 'Num female death',
       'Num male infection', 'Num male death'],
      dtype='object')

In [10]:
df_agg.groupby('country').mean()[['% female infection', '% male infection','% female death','% male death']]

,% female infection,% male infection,% female death,% male death
country,,,,
Afghanistan,48.65,51.35,34.82,65.18
Albania,49.13,50.87,34.75,65.25
Argentina,51.22,48.78,42.50,57.66
Aruba,52.29,47.71,45.03,54.97
Australia,50.20,49.80,42.99,57.01
...,...,...,...,...
Ukraine,53.67,46.33,54.18,45.82
Venezuela,50.82,49.18,39.27,60.73
Vietnam,50.30,49.70,56.98,43.02


In [11]:
df.groupby('country').mean()
    

,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,Num male infection,Total infection,Total Death,% female infection,% male infection,% female death,% male death,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total
country,,,,,,,,,,,,,,,,,,,
Afghanistan,45.000000,167.750000,305.555556,572.000000,3010.000000,3.010000e+06,3176.666667,3.176667e+06,6.186667e+06,877.555556,50.285556,49.714444,35.780000,64.220000,0.111111,0.333333,0.333333,0.111111,0.111111
Albania,38.928571,109.357143,93.733333,176.000000,188.400000,1.884000e+05,195.066667,1.950667e+05,3.834667e+05,269.733333,49.526000,50.474000,32.470667,40.862667,0.266667,0.266667,0.333333,0.066667,0.066667
Argentina,40.000000,97.823529,5630.666667,7639.000000,2572.000000,2.572000e+06,2449.777778,2.449778e+06,5.021778e+06,13258.666667,52.572222,47.427778,42.486111,57.522778,0.222222,0.333333,0.333333,0.055556,0.055556
Aruba,40.000000,97.823529,7.555556,9.222222,6.333333,6.333333e+03,5.777778,5.777778e+03,1.211111e+04,16.777778,53.081667,46.918333,29.073889,32.037222,0.222222,0.333333,0.333333,0.055556,0.055556
Australia,45.000000,144.000000,659.636364,874.727273,2327.454545,2.327455e+06,2308.727273,2.308727e+06,4.636182e+06,1534.363636,51.991818,48.008182,41.727273,58.272727,0.181818,0.272727,0.272727,0.181818,0.090909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ukraine,38.928571,109.357143,7690.400000,6504.000000,3129.600000,3.129600e+06,2701.866667,2.701867e+06,5.831467e+06,14194.400000,55.218000,44.782000,57.448667,42.551333,0.266667,0.266667,0.333333,0.066667,0.066667
Venezuela,40.000000,97.823529,26.222222,40.555556,1605.777778,1.605778e+06,1554.000000,1.554000e+06,3.159778e+06,66.777778,52.018333,47.981667,39.746471,60.253529,0.222222,0.333333,0.333333,0.055556,0.055556
Vietnam,45.000000,144.000000,8.909091,6.727273,8862.000000,8.862000e+06,8755.090909,8.755091e+06,1.761709e+07,15.636364,54.716364,45.283636,64.973333,35.026667,0.181818,0.272727,0.272727,0.181818,0.090909


In [12]:
#basic for loop to get info for each row
for index, row in df_agg.iterrows():
    country = row['country']
    femaleInfect = row['% female infection']
    femaleDeath = row['% female death']
    maleInfect = row['% male infection']
    maleDeath = row['% male death']
    print(f'{country} had {femaleInfect}% and {femaleDeath}% female deaths')
    print(f'{country} had {maleInfect}% and {maleDeath}% male deaths')


Afghanistan had 48.65% and 34.82% female deaths
Afghanistan had 51.35% and 65.18% male deaths
Albania had 49.13% and 34.75% female deaths
Albania had 50.87% and 65.25% male deaths
Argentina had 51.22% and 42.5% female deaths
Argentina had 48.78% and 57.66% male deaths
Aruba had 52.29% and 45.03% female deaths
Aruba had 47.71% and 54.97% male deaths
Australia had 50.2% and 42.99% female deaths
Australia had 49.8% and 57.01% male deaths
Austria had 50.69% and 47.08% female deaths
Austria had 49.31% and 52.92% male deaths
Bahamas had 51.15% and 50.16% female deaths
Bahamas had 48.85% and 50.71% male deaths
Bahrain had 28.36% and 22.28% female deaths
Bahrain had 52.18% and 39.05% male deaths
Bangladesh had 48.98% and 30.32% female deaths
Bangladesh had 49.62% and 69.41% male deaths
Barbados had 54.58% and 56.25% female deaths
Barbados had 50.92% and 43.75% male deaths
Belgium had 50.43% and 47.9% female deaths
Belgium had 49.57% and 52.1% male deaths
Bosnia and Herzegovina had 51.01% and 4

In [13]:
df_agg['Num female survive'] = df_agg['Num female infection'] - df_agg['Num female death']
df_agg['Num male survive'] = df_agg['Num male infection'] - df_agg['Num male death']


## Risk Ratios of Male vs. Female death per country

In [14]:
df_test = df_agg[df_agg['Num male infection']!= 0]
dict = {'Country':[],'Female Infections':[],'Female Deaths':[],'Male Infections':[],'Male Deaths':[],
           'Risk Ratio (Male)':[],'Signifigance':[],'Lower CI':[],'Upper CI':[]}
dict2 = {'Country':[],'Female Infections':[],'Female Deaths':[],'Male Infections':[],'Male Deaths':[],
           'Risk Ratio (Male)':[],'Signifigance':[],'Lower CI':[],'Upper CI':[]}
agg  = pd.DataFrame(dict)

for index, row in df_test.iterrows():
    # pull numbers
    country = row['country']
    femaleInfect = row['Num female infection']
    femaleDeath = row['Num female death']
    femaleSurvive = row['Num female survive']
    maleInfect = row['Num male infection']
    maleDeath = row['Num male death']
    maleSurvive = row['Num male survive']
    
    # Calculate Relative Risk and CIs
    totalMale = maleDeath + maleSurvive
    totalFemale = femaleDeath + femaleSurvive
    maleRisk = maleDeath/totalMale
    femaleRisk = femaleDeath/totalFemale
    riskRatio = maleRisk/femaleRisk
    riskSE = sqrt((1/maleDeath)+(1/femaleDeath)-(1/(maleDeath+maleSurvive))-(1/(femaleDeath+femaleSurvive)))
    lowerCI = exp(log(riskRatio)-1.96*riskSE)
    upperCI = exp(log(riskRatio)+1.96*riskSE)
    
    
    # Chi-Square testing
    cross = pd.DataFrame({'Died':[maleDeath, femaleDeath],
                         'Survived':[maleSurvive, femaleSurvive]})
    cross.index = ['Male infections','Female infections'] 
    c, p_val, dof, expected = chi2_contingency(cross)
    sig = get_asterisks_for_pval(p_val)
    # add to dataframe
    agg.loc[len(agg.index)] = [country,femaleInfect,femaleDeath,maleInfect,maleDeath,riskRatio,sig,lowerCI,upperCI]

agg
# agg.to_csv('Aggregate Signifigance Testing.csv')

,Country,Female Infections,Female Deaths,Male Infections,Male Deaths,Risk Ratio (Male),Signifigance,Lower CI,Upper CI
0,Afghanistan,13544160,1375,14295840,2574,1.773570,****,1.661180,1.893563
1,Albania,1412979,703,1463021,1320,1.813442,****,1.654902,1.987171
2,Argentina,23149391,50672,22046609,68747,1.424569,****,1.408335,1.440990
3,Aruba,56996,68,52004,83,1.337756,ns,0.971050,1.842943
4,Australia,12800498,3628,12698502,4811,1.336726,****,1.280350,1.395585
...,...,...,...,...,...,...,...,...,...
96,Uganda,15837315,112,15046685,252,2.368227,****,1.895640,2.958629
97,Ukraine,23473111,57679,20262889,48779,0.979680,***,0.967954,0.991549
98,Venezuela,14452192,236,13985808,365,1.598185,****,1.356834,1.882466
99,Vietnam,48737682,49,48156318,37,0.764218,ns,0.498684,1.171141


In [15]:
agg.drop(agg.index, inplace=True)

In [16]:
country = df_agg[(df_agg['country']=='Afghanistan')]
country


,country,deathsF,deathsM,populationin1000sF,populationin1000sM,Total infection,Total Death,% female infection,% male infection,% female death,% male death,Num female infection,Num female death,Num male infection,Num male death,Num female survive,Num male survive
0,Afghanistan,1375,2574,13545,14295,27840000,3949,48.65,51.35,34.82,65.18,13544160,1375,14295840,2574,13542785,14293266


In [17]:
df_agg['Num female infection'] = df_agg['Num female infection'].astype(float)
df_agg['Num female death'] = df_agg['Num female death'].astype(float)
df_agg['Num male infection'] = df_agg['Num male infection'].astype(float)
df_agg['Num male death'] = df_agg['Num male death'].astype(float)
df_agg.dtypes

country                  object
deathsF                   int64
deathsM                   int64
populationin1000sF        int64
populationin1000sM        int64
Total infection           int64
Total Death               int64
% female infection      float64
% male infection        float64
% female death          float64
% male death            float64
Num female infection    float64
Num female death        float64
Num male infection      float64
Num male death          float64
Num female survive        int64
Num male survive          int64
dtype: object

In [18]:
country = df_agg[(df_agg['country']=='Afghanistan')]
print(country['Num female infection'])
chisq, p = chisquare([country['Num female infection'], country['Num male infection']])
print(chisq,p)

0    13544160.0
Name: Num female infection, dtype: float64
[20295.36] [0.]


In [19]:
country['Num male infection']


0    14295840.0
Name: Num male infection, dtype: float64

In [20]:
# df_agg[(df_agg['country']=='Afghanistan')]
# x= 
# contigency= pd.crosstab(dff[x], dff[y]).round(3)
# print(contigency)
# # start by running chi2 test on the matrix
# chi2, p, dof, ex = chi2_contingency(contigency, correction=True)
# print(f"Chi2 result of the IAV contingency table: {chi2}, p-value: {p} ({get_asterisks_for_pval(p)})")
# print("")

In [21]:
df.columns

Index(['country', 'date', 'age_group', 'age_begin', 'age_end', 'deathsF',
       'deathsM', 'populationin1000sF', 'Num female infection',
       'populationin1000sM', 'Num male infection', 'Total infection',
       'Total Death', '% female infection', '% male infection',
       '% female death', '% male death', 'Country begin', 'Bin 0-19',
       'Bin 20-49', 'Bin 50-79', 'Bin 80+', 'Total'],
      dtype='object')

In [22]:
df

,country,date,age_group,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,...,% female infection,% male infection,% female death,% male death,Country begin,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total
0,Afghanistan,11/16/2021,19-Oct,10.0,19.0,13.0,12.0,4792.0,4792000,5030.0,...,48.79,51.21,52.00,48.00,Afghanistan,1,0,0,0,0
1,Afghanistan,11/16/2021,20-29,20.0,29.0,41.0,66.0,3408.0,3408000,3628.0,...,48.44,51.56,38.32,61.68,NaN,0,1,0,0,0
2,Afghanistan,11/16/2021,30-39,30.0,39.0,55.0,142.0,2167.0,2167000,2367.0,...,47.79,52.21,27.92,72.08,NaN,0,1,0,0,0
3,Afghanistan,11/16/2021,40-49,40.0,49.0,141.0,291.0,1418.0,1418000,1545.0,...,47.86,52.14,32.64,67.36,NaN,0,1,0,0,0
4,Afghanistan,11/16/2021,50-59,50.0,59.0,318.0,472.0,900.0,900000,941.0,...,48.89,51.11,40.25,59.75,NaN,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Wales,6/16/2022,45-64,45.0,64.0,413.0,612.0,426.0,426000,403.0,...,51.39,48.61,40.29,59.71,NaN,0,0,0,0,0
1459,Wales,6/16/2022,65-74,65.0,74.0,650.0,1064.0,187.0,187000,175.0,...,51.66,48.34,37.92,62.08,NaN,0,0,1,0,0
1460,Wales,6/16/2022,75-84,75.0,84.0,1349.0,1971.0,120.0,120000,102.0,...,54.05,45.95,40.63,59.37,NaN,0,0,0,0,0
1461,Wales,6/16/2022,85+,85.0,999.0,2244.0,1823.0,53.0,53000,32.0,...,62.35,37.65,55.18,44.82,NaN,0,0,0,1,0


## Countries that don't fit into bins

In [23]:
# Finding countries that do not fit into the bins

binLess= df[(df['Bin 0-19']!=1)
              & (df['Bin 20-49']!=1)
              & (df['Bin 50-79']!=1)
              & (df['Bin 80+']!=1)
              & (df['Total']!=1)]
                 
# test = df.drop(df[((df['Bin 0-19']==1) | (df['Bin 20-49']==1) | (df['Bin 50-79']==1) | (df['Bin 80+']==1)) 
#                   | (df['Country begin']!='')].index)

display(binLess)
removeCountry = binLess['country'].unique()
display(removeCountry)
rClist = removeCountry.tolist()
binLess['country'].nunique()
# for country in binLess:
#     print(binLess['country'].unique())


,country,date,age_group,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,...,% female infection,% male infection,% female death,% male death,Country begin,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total
76,Austria,6/15/2022,45-54,45.0,54.0,137.0,261.0,663.0,663000,663.0,...,50.00,50.00,34.42,65.58,NaN,0,0,0,0,0
79,Austria,6/15/2022,75-84,75.0,84.0,2754.0,3860.0,368.0,368000,276.0,...,57.14,42.86,41.64,58.36,NaN,0,0,0,0,0
119,Bangladesh,7/26/2021,45-54,45.0,54.0,53942.0,86906.0,8748.0,8748000,8818.0,...,49.80,50.20,38.30,61.70,NaN,0,0,0,0,0
122,Bangladesh,7/26/2021,75-84,75.0,84.0,34962.0,110880.0,1417.0,1417000,1314.0,...,51.89,48.11,23.97,76.03,NaN,0,0,0,0,0
145,Belgium,5/12/2022,45-64,45.0,64.0,796.0,1540.0,1532.0,1532000,1569.0,...,49.40,50.60,34.08,65.92,NaN,0,0,0,0,0
147,Belgium,5/12/2022,75-84,75.0,84.0,4014.0,5359.0,401.0,401000,304.0,...,56.88,43.12,42.83,57.17,NaN,0,0,0,0,0
154,Bosnia and Herzegovina,6/13/2022,41-50,41.0,50.0,64.0,144.0,221.0,221000,226.0,...,49.44,50.56,30.77,69.23,NaN,0,0,0,0,0
157,Bosnia and Herzegovina,6/13/2022,71-80,71.0,80.0,1324.0,1671.0,133.0,133000,93.0,...,58.85,41.15,44.21,55.79,NaN,0,0,0,0,0
254,Chad,5/16/2021,45-59,45.0,59.0,6.0,30.0,555.0,555000,533.0,...,51.01,48.99,16.67,83.33,NaN,0,0,0,0,0
255,Chad,5/16/2021,60+,60.0,999.0,10.0,50.0,339.0,339000,289.0,...,53.98,46.02,16.67,83.33,NaN,0,0,0,0,0


array(['Austria', 'Bangladesh', 'Belgium', 'Bosnia and Herzegovina',
       'Chad', 'Equatorial Guinea', 'Germany', 'Greece', 'Guinea-Bissau',
       'Jordan', 'Kenya', 'Maldives', 'Myanmar', 'Nepal',
       'Northern Ireland', 'Scotland', 'Wales'], dtype=object)

17

In [24]:
#Removing countries that do not fit in the bins
df_Grp = df.groupby(df['country'])
df_clean = df[~df['country'].isin(rClist)]
df_cleanGrp = df_clean.groupby(df_clean['country'])
display(df_clean)
display(df_clean.nunique()['country'])
display(df.nunique()['country'])
difference = (df.nunique()['country']) - (df_clean.nunique()['country'])
display(difference)

,country,date,age_group,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,...,% female infection,% male infection,% female death,% male death,Country begin,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total
0,Afghanistan,11/16/2021,19-Oct,10.0,19.0,13.0,12.0,4792.0,4792000,5030.0,...,48.79,51.21,52.00,48.00,Afghanistan,1,0,0,0,0
1,Afghanistan,11/16/2021,20-29,20.0,29.0,41.0,66.0,3408.0,3408000,3628.0,...,48.44,51.56,38.32,61.68,NaN,0,1,0,0,0
2,Afghanistan,11/16/2021,30-39,30.0,39.0,55.0,142.0,2167.0,2167000,2367.0,...,47.79,52.21,27.92,72.08,NaN,0,1,0,0,0
3,Afghanistan,11/16/2021,40-49,40.0,49.0,141.0,291.0,1418.0,1418000,1545.0,...,47.86,52.14,32.64,67.36,NaN,0,1,0,0,0
4,Afghanistan,11/16/2021,50-59,50.0,59.0,318.0,472.0,900.0,900000,941.0,...,48.89,51.11,40.25,59.75,NaN,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,Vietnam,8/8/2021,60-69,60.0,69.0,14.0,14.0,3991.0,3991000,3150.0,...,55.89,44.11,50.00,50.00,NaN,0,0,1,0,0
1453,Vietnam,8/8/2021,70-79,70.0,79.0,5.0,5.0,1691.0,1691000,1028.0,...,62.19,37.81,50.00,50.00,NaN,0,0,1,0,0
1454,Vietnam,8/8/2021,80-89,80.0,89.0,11.0,7.0,979.0,979000,498.0,...,66.28,33.72,61.11,38.89,NaN,0,0,0,1,0
1455,Vietnam,8/8/2021,90+,90.0,999.0,2.0,1.0,286.0,286000,111.0,...,72.04,27.96,66.67,33.33,NaN,0,0,0,1,0


90

107

17

### Country lists

In [194]:
removeCountry = df_clean['country'].unique()
countryList = df['country'].unique()
fullCountrylist = countryList.tolist()
cleanedCountrylist = removeCountry.tolist()

In [195]:
fullCountrylist
cleanedCountrylist

['Afghanistan',
 'Albania',
 'Argentina',
 'Aruba',
 'Australia',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Brazil',
 'Brunei Darussalam',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Chile',
 'chile',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Curaçao',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'England',
 'Estonia',
 'Eswatini',
 'Finland',
 'France',
 'Guatemala',
 'Haiti',
 'Hong Kong',
 'Hungary',
 'Iceland',
 'Indonesia',
 'Iraq',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jersey',
 'Kosovo',
 'Kyrgyzstan',
 'Latvia',
 'Liechtenstein',
 'Lithuania',
 'Luxembourg',
 'Malaysia',
 'Malta',
 'Mexico',
 'Moldova',
 'Mongolia',
 'Montenegro',
 'Namibia',
 'Netherlands',
 'Nigeria',
 'North Macedonia',
 'Norway',
 'Oman',
 'Pakistan',
 'Palestine',
 'Panama',
 'Peru',
 'Philippines',
 'Poland',
 'Portugal',
 'Qatar',
 'Republic of Ireland',
 'Romania',
 'Saint Kitts and Nevis',
 'Singapore',
 'Sint Maarten',
 'Slovakia',
 'Slovenia',
 

# Difference in Age Strata Bins

In [27]:
df_bin1 = df.loc[df['Bin 0-19']==1]
df_bin2 = df.loc[df['Bin 20-49']==1]
df_bin3 = df.loc[df['Bin 50-79']==1]
df_bin4 = df.loc[df['Bin 80+']==1]

df_bin1c = df_clean.loc[df_clean['Bin 0-19']==1]
df_bin2c = df_clean.loc[df_clean['Bin 20-49']==1]
df_bin3c = df_clean.loc[df_clean['Bin 50-79']==1]
df_bin4c = df_clean.loc[df_clean['Bin 80+']==1]

In [28]:
filt = ['country','Num female infection','deathsF','Num male infection','deathsM',
        '% female infection','% male infection','% female death','% male death']

In [43]:
bin1 = df_bin1.groupby(df['country']).sum()
bin2 = df_bin2.groupby(df['country']).sum()
bin3 = df_bin3.groupby(df['country']).sum()
bin4 = df_bin4.groupby(df['country']).sum()
bin1 = bin1.reset_index()
bin2 = bin2.reset_index()
bin3 = bin3.reset_index()
bin4 = bin4.reset_index()

bin1[filt]

,country,Num female infection,deathsF,Num male infection,deathsM,% female infection,% male infection,% female death,% male death
0,Afghanistan,4792000,13.0,5030000,12.0,48.79,51.21,52.00,48.00
1,Albania,328000,0.0,368000,0.0,188.55,211.45,0.00,0.00
2,Argentina,7160000,176.0,7423000,183.0,196.39,203.61,194.21,205.79
3,Aruba,13000,0.0,13000,0.0,200.00,200.00,0.00,0.00
4,Australia,3137000,7.0,3302000,8.0,97.45,102.55,92.86,107.14
...,...,...,...,...,...,...,...,...,...
100,Uganda,5693000,1.0,5756000,2.0,49.72,50.28,33.33,66.67
101,Ukraine,4352000,48.0,4621000,48.0,193.99,206.01,239.00,161.00
102,Venezuela,5021000,2.0,5233000,4.0,195.88,204.12,83.33,216.67
103,Vietnam,13809000,0.0,15144000,0.0,95.44,104.56,0.00,0.00


In [44]:
bin1['Num female survive'] = bin1['Num female infection'] - bin1['deathsF']
bin1['Num male survive'] = bin1['Num male infection'] - bin1['deathsM']
bin2['Num female survive'] = bin2['Num female infection'] - bin2['deathsF']
bin2['Num male survive'] = bin2['Num male infection'] - bin2['deathsM']
bin3['Num female survive'] = bin3['Num female infection'] - bin3['deathsF']
bin3['Num male survive'] = bin3['Num male infection'] - bin3['deathsM']
bin4['Num female survive'] = bin4['Num female infection'] - bin4['deathsF']
bin4['Num male survive'] = bin4['Num male infection'] - bin4['deathsM']

In [45]:
bin1.loc[bin1['age_begin']==10]
# bin1 = bin1.reset_index()
# bin1['age_begin']

,country,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,Num male infection,Total infection,...,% male infection,% female death,% male death,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total,Num female survive,Num male survive
0,Afghanistan,10.0,19.0,13.0,12.0,4792.0,4792000,5030.0,5030000,9822000,...,51.21,52.00,48.00,1,0,0,0,0,4791987.0,5029988.0
4,Australia,10.0,28.0,7.0,8.0,3137.0,3137000,3302.0,3302000,6439000,...,102.55,92.86,107.14,2,0,0,0,0,3136993.0,3301992.0
15,Cambodia,10.0,28.0,0.0,0.0,3238.0,3238000,3350.0,3350000,6588000,...,101.67,0.00,0.00,2,0,0,0,0,3238000.0,3350000.0
20,Colombia,10.0,28.0,180.0,228.0,7596.0,7596000,7939.0,7939000,15535000,...,102.21,88.14,111.86,2,0,0,0,0,7595820.0,7938772.0
23,Cuba,10.0,28.0,2.0,2.0,1189.0,1189000,1272.0,1272000,2461000,...,103.37,133.33,66.67,2,0,0,0,0,1188998.0,1271998.0
27,Denmark,10.0,28.0,2.0,5.0,625.0,625000,657.0,657000,1282000,...,102.50,58.33,141.67,2,0,0,0,0,624998.0,656995.0
33,Eswatini,10.0,28.0,3.0,5.0,278.0,278000,284.0,284000,562000,...,101.07,42.86,157.14,2,0,0,0,0,277997.0,283995.0
38,Guatemala,10.0,28.0,167.0,182.0,3876.0,3876000,4043.0,4043000,7919000,...,102.11,98.70,101.30,2,0,0,0,0,3875833.0,4042818.0
40,Haiti,10.0,28.0,5.0,10.0,2385.0,2385000,2454.0,2454000,4839000,...,101.41,60.00,140.00,2,0,0,0,0,2384995.0,2453990.0
41,Hong Kong,10.0,28.0,0.0,0.0,579.0,579000,638.0,638000,1217000,...,104.82,0.00,0.00,2,0,0,0,0,579000.0,638000.0


In [46]:
df_bin1

,country,date,age_group,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,...,% female infection,% male infection,% female death,% male death,Country begin,Bin 0-19,Bin 20-49,Bin 50-79,Bin 80+,Total
0,Afghanistan,11/16/2021,19-Oct,10.0,19.0,13.0,12.0,4792.0,4792000,5030.0,...,48.79,51.21,52.00,48.00,Afghanistan,1,0,0,0,0
9,Albania,3/21/2022,0-4,0.0,4.0,0.0,0.0,80.0,80000,87.0,...,47.90,52.10,0.00,0.00,Albania,1,0,0,0,0
10,Albania,3/21/2022,9-May,5.0,9.0,0.0,0.0,80.0,80000,88.0,...,47.62,52.38,0.00,0.00,NaN,1,0,0,0,0
11,Albania,3/21/2022,14-Oct,10.0,14.0,0.0,0.0,75.0,75000,87.0,...,46.30,53.70,0.00,0.00,NaN,1,0,0,0,0
12,Albania,3/21/2022,15-19,15.0,19.0,0.0,0.0,93.0,93000,106.0,...,46.73,53.27,0.00,0.00,NaN,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,Venezuela,3/14/2022,14-Oct,10.0,14.0,1.0,1.0,1298.0,1298000,1360.0,...,48.83,51.17,50.00,50.00,NaN,1,0,0,0,0
1431,Venezuela,3/14/2022,15-19,15.0,19.0,1.0,2.0,1226.0,1226000,1275.0,...,49.02,50.98,33.33,66.67,NaN,1,0,0,0,0
1446,Vietnam,8/8/2021,0-9,0.0,9.0,0.0,0.0,7309.0,7309000,8140.0,...,47.31,52.69,NaN,NaN,NaN,1,0,0,0,0
1447,Vietnam,8/8/2021,19-Oct,10.0,19.0,0.0,0.0,6500.0,6500000,7004.0,...,48.13,51.87,NaN,NaN,NaN,1,0,0,0,0


In [47]:
# binList = [df_bin1, df_bin2, df_bin3, df_bin4]
binList = [bin1, bin2, bin3, bin4]

In [48]:
#General get data formula
countryinput = 'Vietnam'
pullNum = 'Num female infection'
binNum = bin1
binNum.loc[binNum['country']==countryinput][pullNum]
# getData = binNum.loc[binNum['country']==countryinput][pullNum]
# print(getData)

103    13809000
Name: Num female infection, dtype: int64

In [49]:
agg_strat  = pd.DataFrame(dict)
for country in fullCountrylist:
    countryinput = country
    pullNum = 'Total infection'
    binNum = bin1
    getData = binNum.loc[binNum['country']==countryinput].agg('sum')[pullNum]
#     agg_strat.loc[len(agg_strat.index)] = [country,getData]
    print(countryinput)
    print(getData)
# agg_strat

Afghanistan
9822000
Albania
696000
Argentina
14583000
Aruba
26000
Australia
6439000
Austria
2259000
Bahamas
118000
Bahrain
401000
Bangladesh
75036000
Barbados
68000
Belgium
3272000
Bosnia and Herzegovina
681000
Brazil
60237000
Brunei Darussalam
130000
Cabo Verde
153000
Cambodia
6588000
Cameroon
14023000
Canada
7942000
Chad
10744000
Chile
4922000
chile
0.0
Colombia
15535000
Costa Rica
1426000
Croatia
794000
Cuba
2461000
Curaçao
40000
Cyprus
273000
Czech Republic
2175000
Denmark
1282000
Dominican Republic
3935000
Ecuador
6394000
England
13329000
Equatorial Guinea
1207000
Estonia
280000
Eswatini
562000
Finland
1176000
France
15412000
Germany
30630000
Greece
4344000
Guatemala
7919000
Guinea-Bissau
1275000
Haiti
4839000
Hong Kong
1217000
Hungary
1878000
Iceland
87000
Indonesia
94260000
Iraq
19320000
Israel
3077000
Italy
10729000
Jamaica
931000
Japan
21447000
Jersey
0
Jordan
5333000
Kenya
26760000
Kosovo
587000
Kyrgyzstan
2629000
Latvia
398000
Liechtenstein
0
Lithuania
543000
Luxembourg
1330

In [50]:
agg_strat  = pd.DataFrame(columns = ['Country','Total Infections'])
agg_strat

,Country,Total Infections


In [51]:
bin1[]

SyntaxError: invalid syntax (297467552.py, line 1)

## Building a dataset with all the age stratifaction data

In [39]:
agg_strat_data  = pd.DataFrame(columns = ['Country','Bin','Age Range','Total Infections','Total Deaths',
                                         'Male Infections','Male Deaths','Male Survivors','Female Infections',
                                         'Female Deaths', 'Female Survivors',
                                          'Male Risk Ratio','RR Lower CI','RR Upper CI'])

for country in fullCountrylist:
    warnings.filterwarnings("ignore")
    countryinput = country
    binName = 1
 
    for i in binList:
        binNum = i
        # Total infections
        tInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Total infection']
        # Total deaths
        tDth = binNum.loc[binNum['country']==countryinput].agg('sum')['Total Death']  
        # Male infections
        mInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male infection']
        # Female infections
        fInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female infection']
        # Male deaths
        mDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsM']
        # Male survive
        maleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male survive']
        # Female deaths
        fDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsF']
        # Female survive
        femaleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female survive']
        
        # Male Risk Ratio
        try:   
            maleRisk = mDth/mInf
            femaleRisk = fDth/fInf
            riskRatio = maleRisk/femaleRisk
            riskSE = sqrt((1/mDth)+(1/fDth)-(1/(mDth+maleSurvive))-(1/(fDth+femaleSurvive)))
            RRlowerCI = exp(log(riskRatio)-1.96*riskSE)
            RRupperCI = exp(log(riskRatio)+1.96*riskSE)
        except ValueError:
            riskSE = 'N/A'
            RRlowerCI = 'N/A'
            RRupperCI = 'N/A'
        if binName == 1:
            binAge = '0-19'
        elif binName == 2:
            binAge = '20-49'
        elif binName == 3:
            binAge = '50-79'
        else:
            binAge = '80+'
        
        agg_strat_data.loc[len(agg_strat_data.index)] = [countryinput, binName,binAge, tInf,tDth,mInf,mDth,maleSurvive,
                                                        fInf,fDth,femaleSurvive,riskRatio,RRlowerCI,RRupperCI]
       
        
        
        
        
        
        binName += 1

In [414]:
# agg_strat_data.to_csv('Age Stratification Data.csv')

In [52]:
agg_strat_data

,Country,Bin,Age Range,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
0,Afghanistan,1,0-19,9822000.0,25.0,5030000.0,12.0,5029988.0,4792000.0,13.0,4791987.0,0.879401,0.401262,1.927285
1,Afghanistan,2,20-49,14533000.0,736.0,7540000.0,499.0,7539501.0,6993000.0,237.0,6992763.0,1.952740,1.672994,2.279262
2,Afghanistan,3,50-79,3379000.0,2772.0,1679000.0,1758.0,1677242.0,1700000.0,1014.0,1698986.0,1.755412,1.624895,1.896414
3,Afghanistan,4,80+,106000.0,416.0,46000.0,305.0,45695.0,60000.0,111.0,59889.0,3.584019,2.885098,4.452255
4,Albania,1,0-19,696000.0,0.0,368000.0,0.0,368000.0,328000.0,0.0,328000.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,Vietnam,4,80+,1874000.0,21.0,609000.0,8.0,608992.0,1265000.0,13.0,1264987.0,1.278262,0.52981,3.084035
424,Wales,1,0-19,1144000.0,57.0,581000.0,57.0,580943.0,563000.0,0.0,563000.0,inf,NaN,inf
425,Wales,2,20-49,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
426,Wales,3,50-79,362000.0,1714.0,175000.0,1064.0,173936.0,187000.0,650.0,186350.0,1.749169,1.586904,1.928026


## Same, but run without countries that didn't fit in the categories

In [86]:
agg_strat_data  = pd.DataFrame(columns = ['Country','Bin','Age Range','Total Infections','Total Deaths',
                                         'Male Infections','Male Deaths','Male Survivors','Female Infections',
                                         'Female Deaths', 'Female Survivors',
                                          'Male Risk Ratio','RR Lower CI','RR Upper CI'])

for country in cleanedCountrylist:
    warnings.filterwarnings("ignore")
    countryinput = country
    binName = 1
 
    for i in binList:
        binNum = i
        # Total infections
        tInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Total infection']
        # Total deaths
        tDth = binNum.loc[binNum['country']==countryinput].agg('sum')['Total Death']  
        # Male infections
        mInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male infection']
        # Female infections
        fInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female infection']
        # Male deaths
        mDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsM']
        # Male survive
        maleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male survive']
        # Female deaths
        fDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsF']
        # Female survive
        femaleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female survive']
        
        # Check for zeros that will cause math errors
        if fDth == 0 or mDth == 0 or fInf == 0 or mInf == 0:
            binName += 1
            pass
        else:
            # Male Risk Ratio
            try:   
                maleRisk = mDth/mInf
                femaleRisk = fDth/fInf
                riskRatio = maleRisk/femaleRisk
                riskSE = sqrt((1/mDth)+(1/fDth)-(1/(mDth+maleSurvive))-(1/(fDth+femaleSurvive)))
                RRlowerCI = exp(log(riskRatio)-1.96*riskSE)
                RRupperCI = exp(log(riskRatio)+1.96*riskSE)
            except ValueError:
                riskSE = 'N/A'
                RRlowerCI = 'N/A'
                RRupperCI = 'N/A'
            if binName == 1:
                binAge = '0-19'
            elif binName == 2:
                binAge = '20-49'
            elif binName == 3:
                binAge = '50-79'
            else:
                binAge = '80+'

            agg_strat_data.loc[len(agg_strat_data.index)] = [countryinput, binName,binAge, tInf,tDth,mInf,mDth,maleSurvive,
                                                            fInf,fDth,femaleSurvive,riskRatio,RRlowerCI,RRupperCI]






            binName += 1

In [85]:
agg_strat_data[(agg_strat_data['Country']=='Iceland') | (agg_strat_data['Country']=='Jersey')]
# agg_strat_data
# agg_strat_data.to_csv('Age Stratification Data_cleaned countries_no zeros.csv')

,Country,Bin,Age Range,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
123,Iceland,3,50-79,103000,31,51000,22.0,50978.0,52000,9.0,51991.0,2.492375,1.147737,5.412332
124,Iceland,4,80+,12000,55,5000,29.0,4971.0,7000,26.0,6974.0,1.561538,0.920863,2.647953


# Bin of before and after 30

In [172]:
# 30 Cutoff Dataset, dropping excess rows/columns
df30 = pd.read_csv('Country_age_strat_30.csv', encoding='latin-1')
df30.dropna(how='all', inplace=True)

In [173]:
df30

,country,date,age_group,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,...,Infection,Death,% female infection,% male infection,% female death,% male death,Country begin,Bin 0-30,Bin 30+,Total
0,Afghanistan,11/16/2021,19-Oct,10.0,19.0,13.0,12.0,4792.0,4792000,5030.0,...,9822000,25,48.79,51.21,52.00,48.00,Afghanistan,1.0,0.0,0
1,Afghanistan,11/16/2021,20-29,20.0,29.0,41.0,66.0,3408.0,3408000,3628.0,...,7036000,107,48.44,51.56,38.32,61.68,NaN,1.0,0.0,0
2,Afghanistan,11/16/2021,30-39,30.0,39.0,55.0,142.0,2167.0,2167000,2367.0,...,4534000,197,47.79,52.21,27.92,72.08,NaN,0.0,1.0,0
3,Afghanistan,11/16/2021,40-49,40.0,49.0,141.0,291.0,1418.0,1418000,1545.0,...,2963000,432,47.86,52.14,32.64,67.36,NaN,0.0,1.0,0
4,Afghanistan,11/16/2021,50-59,50.0,59.0,318.0,472.0,900.0,900000,941.0,...,1841000,790,48.89,51.11,40.25,59.75,NaN,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,Wales,6/16/2022,45-64,45.0,64.0,413.0,612.0,426.0,426000,403.0,...,829000,1025,51.39,48.61,40.29,59.71,NaN,0.0,1.0,0
1459,Wales,6/16/2022,65-74,65.0,74.0,650.0,1064.0,187.0,187000,175.0,...,362000,1714,51.66,48.34,37.92,62.08,NaN,0.0,1.0,0
1460,Wales,6/16/2022,75-84,75.0,84.0,1349.0,1971.0,120.0,120000,102.0,...,222000,3320,54.05,45.95,40.63,59.37,NaN,0.0,1.0,0
1461,Wales,6/16/2022,85+,85.0,999.0,2244.0,1823.0,53.0,53000,32.0,...,85000,4067,62.35,37.65,55.18,44.82,NaN,0.0,1.0,0


In [174]:
check = df30[df30['age_begin']==30]
checkList = check['country'].to_list()
checkList = checkList.sort()
cleanedCountrylist = cleanedCountrylist.sort()



In [175]:
# Showing that the same countries do not bin properly
# Note that the bins are not aggregious- some just have 35 as the cutoff instead of 30
if checkList == cleanedCountrylist:
    print('True')
else:
    print('Not true')
check.agg('count')['country']

True


90

In [176]:
df30.columns

Index(['country', 'date', 'age_group', 'age_begin', 'age_end', 'deathsF',
       'deathsM', 'populationin1000sF', 'Num female infection',
       'populationin1000sM', 'Num male infection', 'Infection', 'Death',
       '% female infection', '% male infection', '% female death',
       '% male death', 'Country begin', 'Bin 0-30', 'Bin 30+', 'Total'],
      dtype='object')

In [177]:
df30_bin1 = df30.loc[df30['Bin 0-30']==1]
df30_bin2 = df30.loc[df30['Bin 30+']==1]

bin1_30 = df30_bin1.groupby(df['country']).sum()
bin2_30 = df30_bin2.groupby(df['country']).sum()
bin1_30 = bin1_30.reset_index()
bin2_30 = bin2_30.reset_index()


bin1_30[filt]

,country,Num female infection,deathsF,Num male infection,deathsM,% female infection,% male infection,% female death,% male death
0,Afghanistan,8200000,54.0,8658000,78.0,97.23,102.77,90.32,109.68
1,Albania,554000,7.0,614000,2.0,284.36,315.64,160.00,40.00
2,Argentina,10620000,602.0,10961000,631.0,295.27,304.73,292.53,307.47
3,Aruba,21000,0.0,21000,0.0,300.00,300.00,0.00,0.00
4,Australia,4792000,18.0,5023000,22.0,146.47,153.53,136.86,163.14
...,...,...,...,...,...,...,...,...,...
100,USA,62768000,3101.0,65380000,4388.0,146.93,153.07,131.87,168.13
101,Uganda,9729000,9.0,9593000,11.0,100.98,99.02,80.39,119.61
102,Ukraine,6741000,234.0,7131000,99.0,291.51,308.49,398.52,201.48
103,Venezuela,7093000,6.0,7291000,8.0,296.23,303.77,216.66,283.34


In [178]:
bin1_30['Num female survive'] = bin1_30['Num female infection'] - bin1_30['deathsF']
bin1_30['Num male survive'] = bin1_30['Num male infection'] - bin1_30['deathsM']
bin2_30['Num female survive'] = bin2_30['Num female infection'] - bin2_30['deathsF']
bin2_30['Num male survive'] = bin2_30['Num male infection'] - bin2_30['deathsM']

In [179]:
bin1_30.loc[bin1_30['age_begin']==30]
# bin1_30 = bin1_30.reset_index()
# bin1_30['age_begin']

,country,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,Num male infection,Infection,Death,% female infection,% male infection,% female death,% male death,Bin 0-30,Bin 30+,Total,Num female survive,Num male survive
0,Afghanistan,30.0,48.0,54.0,78.0,8200.0,8200000,8658.0,8658000,16858000,132,97.23,102.77,90.32,109.68,2.0,0.0,0,8199946.0,8657922.0
4,Australia,30.0,57.0,18.0,22.0,4792.0,4792000,5023.0,5023000,9815000,40,146.47,153.53,136.86,163.14,3.0,0.0,0,4791982.0,5022978.0
15,Cambodia,30.0,57.0,2.0,3.0,4764.0,4764000,4926.0,4926000,9690000,5,147.52,152.48,40.00,60.00,3.0,0.0,0,4763998.0,4925997.0
20,Colombia,30.0,57.0,852.0,1176.0,11961.0,11961000,12421.0,12421000,24382000,2028,147.13,152.87,129.62,170.38,3.0,0.0,0,11960148.0,12419824.0
23,Cuba,30.0,57.0,6.0,4.0,1877.0,1877000,2021.0,2021000,3898000,10,144.51,155.49,200.00,100.00,3.0,0.0,0,1876994.0,2020996.0
27,Denmark,30.0,57.0,11.0,20.0,1004.0,1004000,1054.0,1054000,2058000,31,146.34,153.66,95.83,204.17,3.0,0.0,0,1003989.0,1053980.0
33,Eswatini,30.0,57.0,5.0,11.0,382.0,382000,393.0,393000,775000,16,147.76,152.24,67.86,232.14,3.0,0.0,0,381995.0,392989.0
38,Guatemala,30.0,57.0,408.0,519.0,5569.0,5569000,5763.0,5763000,11332000,927,147.49,152.51,140.40,159.60,3.0,0.0,0,5568592.0,5762481.0
40,Haiti,30.0,57.0,13.0,20.0,3410.0,3410000,3459.0,3459000,6869000,33,149.08,150.92,104.44,195.56,3.0,0.0,0,3409987.0,3458980.0
41,Hong Kong,30.0,57.0,0.0,0.0,1030.0,1030000,1054.0,1054000,2084000,0,147.20,152.80,0.00,0.00,3.0,0.0,0,1030000.0,1054000.0


In [180]:
df30_bin1['age_begin']

0       10.0
1       20.0
9        0.0
10       5.0
11      10.0
        ... 
1432    20.0
1433    25.0
1446     0.0
1447    10.0
1448    20.0
Name: age_begin, Length: 494, dtype: float64

In [182]:
binList30 = [bin1_30, bin2_30]

In [190]:
bin2_30[bin2_30['country']=='Wales']

,country,age_begin,age_end,deathsF,deathsM,populationin1000sF,Num female infection,populationin1000sM,Num male infection,Infection,Death,% female infection,% male infection,% female death,% male death,Bin 0-30,Bin 30+,Total,Num female survive,Num male survive
105,Wales,285.0,1265.0,4656.0,5527.0,1349.0,1349000,1293.0,1293000,2642000,10183,268.66,231.34,174.02,325.98,0.0,5.0,0,1344344.0,1287473.0


### Full country for 30 bins

In [187]:
agg_strat_data_30  = pd.DataFrame(columns = ['Country','Bin','Age Range','Total Infections','Total Deaths',
                                             'Male Infections','Male Deaths','Male Survivors','Female Infections',
                                             'Female Deaths', 'Female Survivors',
                                              'Male Risk Ratio','RR Lower CI','RR Upper CI'])
    
for country in fullCountrylist:
    warnings.filterwarnings("ignore")
    countryinput = country
    binName = 1
 
    for i in binList30:
        binNum = i
        # Total infections
        tInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Infection']
        # Total deaths
        tDth = binNum.loc[binNum['country']==countryinput].agg('sum')['Death']  
        # Male infections
        mInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male infection']
        # Female infections
        fInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female infection']
        # Male deaths
        mDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsM']
        # Male survive
        maleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male survive']
        # Female deaths
        fDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsF']
        # Female survive
        femaleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female survive']
        
        # Male Risk Ratio
        try:   
            maleRisk = mDth/mInf
            femaleRisk = fDth/fInf
            riskRatio = maleRisk/femaleRisk
            riskSE = sqrt((1/mDth)+(1/fDth)-(1/(mDth+maleSurvive))-(1/(fDth+femaleSurvive)))
            RRlowerCI = exp(log(riskRatio)-1.96*riskSE)
            RRupperCI = exp(log(riskRatio)+1.96*riskSE)
        except ValueError:
            riskSE = 'N/A'
            RRlowerCI = 'N/A'
            RRupperCI = 'N/A'
        if binName == 1:
            binAge = '0-30'
        elif binName == 2:
            binAge = '30+'
        elif binName == 3:
            binAge = '50-79'
        else:
            binAge = '80+'
        
        agg_strat_data_30.loc[len(agg_strat_data_30.index)] = [countryinput, binName,binAge, tInf,tDth,mInf,mDth,maleSurvive,
                                                                fInf,fDth,femaleSurvive,riskRatio,RRlowerCI,RRupperCI]
       
        
        
        
        
        
        binName += 1

In [191]:
# agg_strat_data_30.to_csv('Age Stratification Data_30.csv')
agg_strat_data_30

,Country,Bin,Age Range,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
0,Afghanistan,1,0-30,16858000.0,132.0,8658000.0,78.0,8657922.0,8200000.0,54.0,8199946.0,1.368035,0.966959,1.935468
1,Afghanistan,2,30+,10982000.0,3817.0,5637000.0,2496.0,5634504.0,5345000.0,1321.0,5343679.0,1.791602,1.676039,1.915132
2,Albania,1,0-30,1168000.0,9.0,614000.0,2.0,613998.0,554000.0,7.0,553993.0,0.257794,0.053553,1.240977
3,Albania,2,30+,1708000.0,2014.0,849000.0,1318.0,847682.0,859000.0,696.0,858304.0,1.915983,1.747946,2.100174
4,Argentina,1,0-30,21581000.0,1233.0,10961000.0,631.0,10960369.0,10620000.0,602.0,10619398.0,1.015564,0.908264,1.135539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Venezuela,2,30+,14054000.0,587.0,6695000.0,357.0,6694643.0,7359000.0,230.0,7358770.0,1.706116,1.445568,2.013626
210,Vietnam,1,0-30,44593000.0,5.0,23258000.0,1.0,23257999.0,21335000.0,4.0,21334996.0,0.229330,0.025631,2.051871
211,Vietnam,2,30+,52301000.0,81.0,24895000.0,36.0,24894964.0,27406000.0,45.0,27405955.0,0.880691,0.56818,1.36509
212,Wales,1,0-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


## Cleaned country for 30 bins

In [193]:
removeCountry = df_clean['country'].unique()
countryList = df['country'].unique()
fullCountrylist = countryList.tolist()
cleanedCountrylist = removeCountry.tolist()

In [196]:
agg_strat_data_30_c  = pd.DataFrame(columns = ['Country','Bin','Age Range','Total Infections','Total Deaths',
                                             'Male Infections','Male Deaths','Male Survivors','Female Infections',
                                             'Female Deaths', 'Female Survivors',
                                              'Male Risk Ratio','RR Lower CI','RR Upper CI'])
    
for country in cleanedCountrylist:
    warnings.filterwarnings("ignore")
    countryinput = country
    binName = 1
 
    for i in binList30:
        binNum = i
        # Total infections
        tInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Infection']
        # Total deaths
        tDth = binNum.loc[binNum['country']==countryinput].agg('sum')['Death']  
        # Male infections
        mInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male infection']
        # Female infections
        fInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female infection']
        # Male deaths
        mDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsM']
        # Male survive
        maleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male survive']
        # Female deaths
        fDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsF']
        # Female survive
        femaleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female survive']
        
        # Male Risk Ratio
        try:   
            maleRisk = mDth/mInf
            femaleRisk = fDth/fInf
            riskRatio = maleRisk/femaleRisk
            riskSE = sqrt((1/mDth)+(1/fDth)-(1/(mDth+maleSurvive))-(1/(fDth+femaleSurvive)))
            RRlowerCI = exp(log(riskRatio)-1.96*riskSE)
            RRupperCI = exp(log(riskRatio)+1.96*riskSE)
        except ValueError:
            riskSE = 'N/A'
            RRlowerCI = 'N/A'
            RRupperCI = 'N/A'
        if binName == 1:
            binAge = '0-30'
        elif binName == 2:
            binAge = '30+'
        elif binName == 3:
            binAge = '50-79'
        else:
            binAge = '80+'
        
        agg_strat_data_30_c.loc[len(agg_strat_data_30_c.index)] = [countryinput, binName,binAge, tInf,tDth,mInf,mDth,maleSurvive,
                                                                fInf,fDth,femaleSurvive,riskRatio,RRlowerCI,RRupperCI]
       
        
        
        
        
        
        binName += 1

In [199]:
# agg_strat_data_30_c.to_csv('Age Stratification Data_30_cleaned.csv')
agg_strat_data_30_c

,Country,Bin,Age Range,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
0,Afghanistan,1,0-30,16858000.0,132.0,8658000.0,78.0,8657922.0,8200000.0,54.0,8199946.0,1.368035,0.966959,1.935468
1,Afghanistan,2,30+,10982000.0,3817.0,5637000.0,2496.0,5634504.0,5345000.0,1321.0,5343679.0,1.791602,1.676039,1.915132
2,Albania,1,0-30,1168000.0,9.0,614000.0,2.0,613998.0,554000.0,7.0,553993.0,0.257794,0.053553,1.240977
3,Albania,2,30+,1708000.0,2014.0,849000.0,1318.0,847682.0,859000.0,696.0,858304.0,1.915983,1.747946,2.100174
4,Argentina,1,0-30,21581000.0,1233.0,10961000.0,631.0,10960369.0,10620000.0,602.0,10619398.0,1.015564,0.908264,1.135539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,Ukraine,2,30+,29864000.0,106125.0,13133000.0,48681.0,13084319.0,16731000.0,57444.0,16673556.0,1.079625,1.06669,1.092716
176,Venezuela,1,0-30,14384000.0,14.0,7291000.0,8.0,7290992.0,7093000.0,6.0,7092994.0,1.297124,0.450062,3.738447
177,Venezuela,2,30+,14054000.0,587.0,6695000.0,357.0,6694643.0,7359000.0,230.0,7358770.0,1.706116,1.445568,2.013626
178,Vietnam,1,0-30,44593000.0,5.0,23258000.0,1.0,23257999.0,21335000.0,4.0,21334996.0,0.229330,0.025631,2.051871


## Full country, drop all zeros

In [207]:
agg_strat_data_30_nozero  = pd.DataFrame(columns = ['Country','Bin','Age Range','Total Infections','Total Deaths',
                                                 'Male Infections','Male Deaths','Male Survivors','Female Infections',
                                                 'Female Deaths', 'Female Survivors',
                                                  'Male Risk Ratio','RR Lower CI','RR Upper CI'])

for country in fullCountrylist:
    warnings.filterwarnings("ignore")
    countryinput = country
    binName = 1
 
    for i in binList30:
        binNum = i
        # Total infections
        tInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Infection']
        # Total deaths
        tDth = binNum.loc[binNum['country']==countryinput].agg('sum')['Death']  
        # Male infections
        mInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male infection']
        # Female infections
        fInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female infection']
        # Male deaths
        mDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsM']
        # Male survive
        maleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male survive']
        # Female deaths
        fDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsF']
        # Female survive
        femaleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female survive']
        
        # Check for zeros that will cause math errors
        if fDth == 0 or mDth == 0 or fInf == 0 or mInf == 0:
            binName += 1
            pass
        else:
            # Male Risk Ratio
            try:   
                maleRisk = mDth/mInf
                femaleRisk = fDth/fInf
                riskRatio = maleRisk/femaleRisk
                riskSE = sqrt((1/mDth)+(1/fDth)-(1/(mDth+maleSurvive))-(1/(fDth+femaleSurvive)))
                RRlowerCI = exp(log(riskRatio)-1.96*riskSE)
                RRupperCI = exp(log(riskRatio)+1.96*riskSE)
            except ValueError:
                riskSE = 'N/A'
                RRlowerCI = 'N/A'
                RRupperCI = 'N/A'
            if binName == 1:
                binAge = '0-30'
            elif binName == 2:
                binAge = '30+'
            elif binName == 3:
                binAge = '50-79'
            else:
                binAge = '80+'

            agg_strat_data_30_nozero.loc[len(agg_strat_data_30_nozero.index)] = [countryinput, binName,binAge, tInf,tDth,
                                                                                 mInf,mDth,maleSurvive,fInf,fDth,femaleSurvive,
                                                                                 riskRatio,RRlowerCI,RRupperCI]


            binName += 1

In [210]:
# agg_strat_data_30_nozero.to_csv('Age Stratification Data_30_No zero.csv')
agg_strat_data_30_nozero

,Country,Bin,Age Range,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
0,Afghanistan,1,0-30,16858000,132,8658000,78.0,8657922.0,8200000,54.0,8199946.0,1.368035,0.966959,1.935468
1,Afghanistan,2,30+,10982000,3817,5637000,2496.0,5634504.0,5345000,1321.0,5343679.0,1.791602,1.676039,1.915132
2,Albania,1,0-30,1168000,9,614000,2.0,613998.0,554000,7.0,553993.0,0.257794,0.053553,1.240977
3,Albania,2,30+,1708000,2014,849000,1318.0,847682.0,859000,696.0,858304.0,1.915983,1.747946,2.100174
4,Argentina,1,0-30,21581000,1233,10961000,631.0,10960369.0,10620000,602.0,10619398.0,1.015564,0.908264,1.135539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Venezuela,1,0-30,14384000,14,7291000,8.0,7290992.0,7093000,6.0,7092994.0,1.297124,0.450062,3.738447
187,Venezuela,2,30+,14054000,587,6695000,357.0,6694643.0,7359000,230.0,7358770.0,1.706116,1.445568,2.013626
188,Vietnam,1,0-30,44593000,5,23258000,1.0,23257999.0,21335000,4.0,21334996.0,0.229330,0.025631,2.051871
189,Vietnam,2,30+,52301000,81,24895000,36.0,24894964.0,27406000,45.0,27405955.0,0.880691,0.568180,1.365090


## Full countr, no zero, bin 30

In [211]:
agg_strat_data_30_nozero_c  = pd.DataFrame(columns = ['Country','Bin','Age Range','Total Infections','Total Deaths',
                                                 'Male Infections','Male Deaths','Male Survivors','Female Infections',
                                                 'Female Deaths', 'Female Survivors',
                                                  'Male Risk Ratio','RR Lower CI','RR Upper CI'])

for country in cleanedCountrylist:
    warnings.filterwarnings("ignore")
    countryinput = country
    binName = 1
 
    for i in binList30:
        binNum = i
        # Total infections
        tInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Infection']
        # Total deaths
        tDth = binNum.loc[binNum['country']==countryinput].agg('sum')['Death']  
        # Male infections
        mInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male infection']
        # Female infections
        fInf = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female infection']
        # Male deaths
        mDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsM']
        # Male survive
        maleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num male survive']
        # Female deaths
        fDth = binNum.loc[binNum['country']==countryinput].agg('sum')['deathsF']
        # Female survive
        femaleSurvive = binNum.loc[binNum['country']==countryinput].agg('sum')['Num female survive']
        
        # Check for zeros that will cause math errors
        if fDth == 0 or mDth == 0 or fInf == 0 or mInf == 0:
            binName += 1
            pass
        else:
            # Male Risk Ratio
            try:   
                maleRisk = mDth/mInf
                femaleRisk = fDth/fInf
                riskRatio = maleRisk/femaleRisk
                riskSE = sqrt((1/mDth)+(1/fDth)-(1/(mDth+maleSurvive))-(1/(fDth+femaleSurvive)))
                RRlowerCI = exp(log(riskRatio)-1.96*riskSE)
                RRupperCI = exp(log(riskRatio)+1.96*riskSE)
            except ValueError:
                riskSE = 'N/A'
                RRlowerCI = 'N/A'
                RRupperCI = 'N/A'
            if binName == 1:
                binAge = '0-30'
            elif binName == 2:
                binAge = '30+'
            elif binName == 3:
                binAge = '50-79'
            else:
                binAge = '80+'

            agg_strat_data_30_nozero_c.loc[len(agg_strat_data_30_nozero_c.index)] = [countryinput, binName,binAge, tInf,tDth,
                                                                                 mInf,mDth,maleSurvive,fInf,fDth,femaleSurvive,
                                                                                 riskRatio,RRlowerCI,RRupperCI]


            binName += 1

In [214]:
# agg_strat_data_30_nozero_c.to_csv('Age Stratification Data_30_No zero_cleaned.csv')
agg_strat_data_30_nozero_c

,Country,Bin,Age Range,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
0,Afghanistan,1,0-30,16858000,132,8658000,78.0,8657922.0,8200000,54.0,8199946.0,1.368035,0.966959,1.935468
1,Afghanistan,2,30+,10982000,3817,5637000,2496.0,5634504.0,5345000,1321.0,5343679.0,1.791602,1.676039,1.915132
2,Albania,1,0-30,1168000,9,614000,2.0,613998.0,554000,7.0,553993.0,0.257794,0.053553,1.240977
3,Albania,2,30+,1708000,2014,849000,1318.0,847682.0,859000,696.0,858304.0,1.915983,1.747946,2.100174
4,Argentina,1,0-30,21581000,1233,10961000,631.0,10960369.0,10620000,602.0,10619398.0,1.015564,0.908264,1.135539
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Ukraine,2,30+,29864000,106125,13133000,48681.0,13084319.0,16731000,57444.0,16673556.0,1.079625,1.066690,1.092716
155,Venezuela,1,0-30,14384000,14,7291000,8.0,7290992.0,7093000,6.0,7092994.0,1.297124,0.450062,3.738447
156,Venezuela,2,30+,14054000,587,6695000,357.0,6694643.0,7359000,230.0,7358770.0,1.706116,1.445568,2.013626
157,Vietnam,1,0-30,44593000,5,23258000,1.0,23257999.0,21335000,4.0,21334996.0,0.229330,0.025631,2.051871


In [215]:
check = agg_strat_data_30_nozero_c

## Aggregating bins

In [220]:
bingroups = agg_strat_data_30_nozero_c
bingroups.columns

Index(['Country', 'Bin', 'Age Range', 'Total Infections', 'Total Deaths',
       'Male Infections', 'Male Deaths', 'Male Survivors', 'Female Infections',
       'Female Deaths', 'Female Survivors', 'Male Risk Ratio', 'RR Lower CI',
       'RR Upper CI'],
      dtype='object')

In [222]:
bingroups.groupby('Bin').sum()

,Total Infections,Total Deaths,Male Infections,Male Deaths,Male Survivors,Female Infections,Female Deaths,Female Survivors,Male Risk Ratio,RR Lower CI,RR Upper CI
Bin,,,,,,,,,,,
1,1282571000,40878,655366000,22413.0,655343587.0,627205000,18465.0,627186535.0,74.266249,43.126191,180.603166
2,1515045000,3370618,733977000,1908374.0,732068626.0,781068000,1462244.0,779605756.0,122.451725,110.262906,138.271316


In [226]:
bingroups.groupby('Bin').sum().to_clipboard()

In [224]:
mDth1 = 22413
mInf1 = 655366000
fDth1 = 18465
fInf1 = 627205000
maleRisk1 = mDth1/mInf1
femaleRisk1 = fDth1/fInf1
maleSurvive1 = 655343587
femaleSurvive1 = 627186535


riskRatio1 = maleRisk1/femaleRisk1
riskSE1 = sqrt((1/mDth1)+(1/fDth1)-(1/(mDth1+maleSurvive1))-(1/(fDth1+femaleSurvive1)))
RRlowerCI1 = exp(log(riskRatio1)-1.96*riskSE1)
RRupperCI1 = exp(log(riskRatio1)+1.96*riskSE1)

print(riskRatio1, " ", RRlowerCI1, " ", RRupperCI1, " ")

1.1616526414309893   1.1392436302927216   1.184502439567615  


In [225]:
mDth2 = 1908374
mInf2 = 733977000
fDth2 = 1462244
fInf2 = 781068000
maleRisk2 = mDth2/mInf2
femaleRisk2 = fDth2/fInf2
maleSurvive2 = 732068626
femaleSurvive2 = 779605756


riskRatio2 = maleRisk2/femaleRisk2
riskSE2 = sqrt((1/mDth2)+(1/fDth2)-(1/(mDth2+maleSurvive2))-(1/(fDth2+femaleSurvive2)))
RRlowerCI2 = exp(log(riskRatio2)-1.96*riskSE2)
RRupperCI2 = exp(log(riskRatio2)+1.96*riskSE2)

print(riskRatio2, " ", RRlowerCI2, " ", RRupperCI2, " ")

1.3888330323554705   1.3858478117904767   1.3918246833104004  
